# Uniquely Identifying Particles

For some applications, it is useful to keep track of which particle is which, and this can get jumbled up when particles are added or removed from the simulation, when they get reindexed if using the gravity tree code etc.

In [1]:
import rebound
sim = rebound.Simulation()
sim.add(m=1.)
sim.add(a=1.)
sim.add(a=2.)

We can refer to a particle using a custom string as follows:

In [2]:
sim.particles[1].hash = sim.get_particle_hash("earth")

REBOUND uses a hash function to convert "earth" to an effectively random integer, and we store that identifier in the particle's hash field.  We can now access the particle with:

In [3]:
print(sim.get_particle("earth"))

<rebound.Particle object, m=0.0 x=1.0 y=0.0 z=0.0 vx=0.0 vy=1.0 vz=0.0>


rather than the typical

In [4]:
print(sim.particles[1])

<rebound.Particle object, m=0.0 x=1.0 y=0.0 z=0.0 vx=0.0 vy=1.0 vz=0.0>


The advantage of the former is that it will always access the right particle even if the particle moves around in the particle array (due to particle additions, removals reindexing etc.), and probably makes your code more readable!  For convenience you can also add the name when you add the particle to the simulation:

In [5]:
sim.add(a=1000., name="Planet 9")
print(sim.get_particle("Planet 9").a)

999.9999999999998


If you need to assign many hashes and don't want to give them each a name, you can also let the simulation assign them unique hashes by not passing a string to `sim.get_particle_hash` after the fact:

In [6]:
for i in range(3,13):
    sim.add(a=i)
    sim.particles[-1].hash = sim.get_particle_hash()

If we print all the particles' hashes, ones that were not assigned one are 0, ones assigned strings will have random integers, and ones assigned by the simulation will be consecutive (plus a random offset):

In [7]:
for p in sim.particles:
    print(p.hash)

0
1424801690
0
545758163
20659
20660
20661
20662
20663
20664
20665
20666
20667
20668


We can also access particles through their hash:

In [8]:
print(sim.get_particle(hash=1424801690))

<rebound.Particle object, m=0.0 x=1.0 y=0.0 z=0.0 vx=0.0 vy=1.0 vz=0.0>


Only particles that have been assigned a non-zero hash can be accessed with `sim.get_particle`.  If you assign hashes manually and two particles share the same hash, `sim.get_particle` will return the first hit in the `particles` array.